In [1]:
!uv pip install -U git+https://github.com/JignyasAnand/rescript-ast-diff

Using Python 3.12.9 environment at: /Users/sakthi.n/Documents/Opensource/cada-rescript/tvenv
Resolved 8 packages in 1.63s                                         
Prepared 1 package in 177ms                                              
Uninstalled 1 package in 1ms
Installed 1 package in 2ms (from git+https://github.com/tree
 - tree-sitter==0.24.0
 + tree-sitter==0.24.0 (from git+https://github.com/tree-sitter/py-tree-sitter@9c78f3b8d10f81b97fbb2181c9333323d6375480)


In [2]:
!uv pip install git+ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git@code-traverse

Using Python 3.12.9 environment at: /Users/sakthi.n/Documents/Opensource/cada-rescript/tvenv
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
   Updating ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (code-traverse)
    Updated ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git (c6e993245059088eb09f8ae
Resolved 38 packages in 989ms                                        
Audited 38 packages in 0.07ms


In [3]:
from rescript_ast_diff import generate_pr_changes_bitbucket
from rescript_ast_diff.bitbucket import BitBucket 

BASE_URL = "https://bitbucket.juspay.net/rest"
PROJECT_KEY = "JBIZ"
REPO_SLUG = "rescript-euler-dashboard"
AUTH = ("sakthi.n@juspay.in", "BBDC-NDg5ODgwNDM2MzkyOjgy8c70YxFmjQlfjSGQD4895tx5")
HEADERS = {"Accept": "application/json"}
PR_ID = "19971"
bitbucket = BitBucket(BASE_URL, PROJECT_KEY, REPO_SLUG, AUTH, HEADERS)
generate_pr_changes_bitbucket(PR_ID, bitbucket, quiet=True)

LATEST COMMIT - 41256f7b31a489df4cc57dddc4e4ae4f2cf7fe36
OLDEST COMMIT - 7dcd1a179f54f2f93dea9f2a82ea7ce3dfe89296
Changes written to -  ./detailed_changes.json


In [5]:
from jaf.core.llm import AzureGPTLLM
from dotenv import load_dotenv 
import os 

load_dotenv()

AZURE_GPT_DEPLOYMENT_NAME=os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
AZURE_OAI_BASE_URL=os.getenv("AZURE_OAI_BASE_URL")
AZURE_OAI_API_VERSION= os.getenv("AZURE_OAI_API_VERSION")
AZURE_OAI_API_KEY= os.getenv("AZURE_OAI_API_KEY")

llm = AzureGPTLLM(
    deployment_name=AZURE_GPT_DEPLOYMENT_NAME,
    api_base=AZURE_OAI_BASE_URL,
    api_version=AZURE_OAI_API_VERSION,
    api_key=AZURE_OAI_API_KEY
)

In [6]:
from pydantic import BaseModel
from typing import Optional

class Data(BaseModel):
   moduleName: str
   addedFunctions: list = []
   modifiedFunctions: list = []
   deletedFunctions: list = []
   addedTypes: list = []
   modifiedTypes: list = []
   deletedTypes: list = []
   addedExternals: list = []
   modifiedExternals: list = []
   deletedExternals: list = []
   # prompt: Optional[str] = "" 
   testCases: Optional[list] = []


In [9]:
import json

with open('../python/detailed_changes.json', 'r') as file:
    data = json.load(file)

result = []
for d in data:
    result.append(Data.model_validate(d))

In [10]:
systemPrompt = """ 
You are a QA automation assistant. Given code changes to frontend components or functions (added or modified), generate structured test case objects in JSON format.

Each test case must contain:

testCaseId: Unique identifier (e.g., TC001, TC002)

description: One-line summary of what the test verifies

stepsToReproduce: A list of precise UI actions (e.g., "Click button", "Enter value in input")

expectedResult: What the correct outcome should be

⚠️ Steps must be realistic, like how a manual tester would execute them.
❌ Avoid vague steps like "Run function" — describe the UI interaction instead.

Output a JSON array of such test cases.

If there are no test cases (e.g., only deletions), return an empty list [].

Eg Format:
[
  {
    "testCaseId": "TC001",
    "description": "User can successfully search when input is valid",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'apple' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "Search results are displayed for 'apple'"
  },
  {
    "testCaseId": "TC002",
    "description": "Search button does nothing if query is too short",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'ap' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "No search is performed; no request sent"
  }
]
"""

userPrompt = """
Here are the frontend code changes in this pull request.
Please generate test cases in JSON format, following this structure:
[
  {{
    "testCaseId": "TC001",
    "description": "What the test is verifying",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }},
  {{
    "testCaseId": "TC002",
    "description": "What the test is verifying",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }}
  ...
]
Be realistic and user-centric in the steps — describe UI actions like clicking, typing, navigation, etc.
Generate test cases only for added or modified functions or components.
Ignore deleted functions unless they affect user-facing behavior.

Added Functions/Components:
{added}

Modified Functions/Components:
{modified}

Deleted Functions/Components:
{deleted}

Based on the above things 
Return the test cases as a JSON array.
Do not include any extra commentary or explanation — just valid JSON.
"""

In [11]:
import re
def parse(text: str) -> dict:
    cleaned = re.sub(r"```(?:json)?", "", text).strip()
    
    # Try parsing it
    try:
        parsed = json.loads(cleaned)
        return parsed
    except json.JSONDecodeError as e:
        print("❌ Failed to parse JSON:", e)
        return None

In [12]:
from jaf.types import Query 

q = Query()

for res in result:
    print(res.moduleName)
    q.prompt = userPrompt.format(added=res.addedFunctions,modified=res.modifiedFunctions,deleted=res.deletedFunctions)
    q.system_prompt = systemPrompt 

    llm(q)
    parsed_resp = parse(q.response)
    res.testCases = parsed_resp

src/entryPoints/pgIntegration/PgIntegrationApp.res
src/libraries/Monaco.res
src/libraries/MonacoEditor.res
src/libraries/ReactDiffViewer.res
src/screens/PGintegration/AddedFeauture.res
src/screens/PGintegration/CreateNewFeature.res
src/screens/PGintegration/FilledEntryFields.res
src/screens/PGintegration/GenerateResponseViewer.res
src/screens/PGintegration/InitialScreen.res
src/screens/PGintegration/IntegrationChecklist.res
src/screens/PGintegration/IntegrationFlow.res
src/screens/PGintegration/IntegrationRouterTabs.res
src/screens/PGintegration/MultiApiIntegrationFlow.res
src/screens/PGintegration/NewTracker.res
src/screens/PGintegration/PostmanData.res
src/screens/PGintegration/ShowFeature.res
src/screens/PGintegration/ShowPgView.res
src/screens/PGintegration/UploadPGDoc.res
src/libraries/SyntaxHighlighter.res
src/screens/PGintegration/EntryPoint.res
src/screens/PGintegration/IntegrationTableEntities.res
src/screens/PGintegration/IntegrationTables.res
src/screens/PGintegration/Prompt

In [13]:
json_list = [] 

for res in result:
    json_list.append(res.model_dump())

In [14]:
with open("/Users/sakthi.n/Documents/Opensource/cada-rescript/llm_response.json", "w") as f:
    json.dump(json_list, f, indent=4)